In [1]:
import pandas as pd
import os
import numpy as np
import datetime
import matplotlib.pyplot as plt


Bad key "text.kerning_factor" on line 4 in
/opt/anaconda3/envs/bigdatalab_cpu_202101/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
status_df = pd.read_csv("../../Data/filtered_status.csv", parse_dates=['time'])

In [3]:
status_df

,station_id,bikes_available,docks_available,time
0,2,2,25,2013-08-29 12:06:01
1,2,2,25,2013-08-29 12:07:01
2,2,2,25,2013-08-29 12:08:01
3,2,2,25,2013-08-29 12:09:01
4,2,2,25,2013-08-29 12:10:01
...,...,...,...,...
71977905,84,8,7,2015-08-31 23:55:02
71977906,84,8,7,2015-08-31 23:56:01
71977907,84,8,7,2015-08-31 23:57:02
71977908,84,8,7,2015-08-31 23:58:02


In [4]:
status_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71977910 entries, 0 to 71977909
Data columns (total 4 columns):
 #   Column           Dtype         
---  ------           -----         
 0   station_id       int64         
 1   bikes_available  int64         
 2   docks_available  int64         
 3   time             datetime64[ns]
dtypes: datetime64[ns](1), int64(3)
memory usage: 2.1 GB


In [3]:
# add day column to the dataframe
status_df['day'] = status_df['time'].dt.date

In [26]:
# calculate how often stations are (almost) empty/full

for ID in [70, 76, 34, 72, 75, 69, 50, 60, 73, 82, 84]: #arbitrary set of stations
    IDdf = status_df[status_df['station_id']==ID]

    length = []
    full = []
    almost_full = []
    normal = []
    almost_empty = []
    empty = []

    for group, df in IDdf.groupby('day'):
        length.append(len(df))
        full.append(len(df[df['docks_available'] == 0]))
        almost_full.append(len(df[df['docks_available'] <= 3]))
        mask = (df['docks_available'] <= 3) | (df['bikes_available'] <= 3)
        normal.append(len(df[~mask]))
        almost_empty.append(len(df[df['bikes_available'] <= 3]))
        empty.append(len(df[df['bikes_available'] == 0]))

    length = np.array(length)
    full = np.array(full)
    almost_full = np.array(almost_full)
    normal = np.array(normal)
    almost_empty = np.array(almost_empty)
    empty = np.array(empty)

    plt.figure(figsize=(10, 6))

    bars = [np.mean(full/length*100), np.mean(almost_full/length*100), np.mean(normal/length*100), np.mean(almost_empty/length*100), np.mean(empty/length*100)]
    names = ['full', 'almost_full', 'normal', 'almost_empty', 'empty']

    plt.bar(names, bars)
    plt.ylabel('Percentage of time')

    if not os.path.exists(f"../../Results/Occupation/"):
        os.makedirs(f"../../Results/Occupation/")
    plt.savefig(f"../../Results/Occupation/Station_{ID}.jpg", bbox_inches='tight')
    plt.close()